In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1635705235221_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
glueContext = GlueContext(SparkContext.getOrCreate())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
mags_df = glueContext.create_dynamic_frame.from_catalog(database="amazon_review", table_name="magazine_subscriptions_json")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
mags_df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- overall: double
|-- vote: string
|-- verified: boolean
|-- reviewTime: string
|-- reviewerID: string
|-- asin: string
|-- reviewerName: string
|-- reviewText: string
|-- summary: string
|-- unixReviewTime: int
|-- style: struct
|    |-- Format:: string
|-- image: array
|    |-- element: string

In [10]:
dfc = mags_df.relationalize("root", "s3://afsana/json/")
dfc.keys()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dict_keys(['root', 'root_image'])

In [19]:
mags_flatdf = dfc.select('root')
mags_flatdf.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- overall: double
|-- vote: string
|-- verified: boolean
|-- reviewTime: string
|-- reviewerID: string
|-- asin: string
|-- reviewerName: string
|-- reviewText: string
|-- summary: string
|-- unixReviewTime: int
|-- style.Format:: string
|-- image: long

In [20]:
mags_flatdf.toDF().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----+--------+-----------+--------------+----------+------------------+--------------------+--------------------+--------------+---------------+-----+
|overall|vote|verified| reviewTime|    reviewerID|      asin|      reviewerName|          reviewText|             summary|unixReviewTime|  style.Format:|image|
+-------+----+--------+-----------+--------------+----------+------------------+--------------------+--------------------+--------------+---------------+-----+
|    5.0|   9|   false| 11 8, 2001| AH2IFH762VY5U|B00005N7P0|      ted sedlmayr|for computer enth...|AVID READER SINCE...|    1005177600|           null| null|
|    5.0|   9|   false|10 31, 2001| AOSFI0JEYU4XM|B00005N7P0|   Amazon Customer|Thank god this is...|  The straight scoop|    1004486400|           null| null|
|    3.0|  14|   false|03 24, 2007|A3JPFWKS83R49V|B00005N7OJ|       Bryan Carey|Antiques Magazine...|Antiques Magazine...|    1174694400| Print Magazine| null|
|    5.0|  13|   false|11 10, 2006|A19FK

In [21]:
mags_flatdf2 = dfc.select('root_image')
mags_flatdf2.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- id: long
|-- index: int
|-- image.val: string

In [22]:
mags_flatdf2.toDF().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+-----+--------------------+
| id|index|           image.val|
+---+-----+--------------------+
|  1|    0|https://images-na...|
|  2|    0|https://images-na...|
|  3|    0|https://images-na...|
|  4|    0|https://images-na...|
|  4|    1|https://images-na...|
|  4|    2|https://images-na...|
|  5|    0|https://images-na...|
|  6|    0|https://images-na...|
|  7|    0|https://images-na...|
|  8|    0|https://images-na...|
|  9|    0|https://images-na...|
| 10|    0|https://images-na...|
| 10|    1|https://images-na...|
| 11|    0|https://images-na...|
| 12|    0|https://images-na...|
| 13|    0|https://images-na...|
| 14|    0|https://images-na...|
| 15|    0|https://images-na...|
| 16|    0|https://images-na...|
| 16|    1|https://images-na...|
+---+-----+--------------------+
only showing top 20 rows

In [23]:
glue_relationalize_output_s3_path = "s3://afsana/json_flat/"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
magsdataoutput = glueContext.write_dynamic_frame.from_options(frame = mags_flatdf, connection_type = "s3", connection_options = {"path": glue_relationalize_output_s3_path}, format = "orc", transformation_ctx = "magsdataoutput")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…